***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import pytz


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu'
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S').dt.tz_localize(tz=pytz.timezone('Asia/Ho_Chi_Minh')))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(lambda x: pd.to_datetime(x).dt.tz_localize(tz='Asia/Ho_Chi_Minh'))

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting['N+'] = transporting['N0']+ pd.Timedelta(hours=12) # Kho lay tai HCM or HN
# transporting['N+'] = transporting['N0'] + pd.Timedelta(hours=32) # Kho lay tai cac tinh con lai
transporting['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn']) | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) &\
             ((data['KhoTra'].isnull() & (data['KhoLay'] != data['KhoHienTai'])) |
              (~data['KhoTra'].isnull() & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=36) # Kho giao tai HCM or HN
# r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=56) # Kho giao tai cac tinh con lai
r_transporting['Aging'] = (datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = 'Others'


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')) - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.now(tz=pytz.timezone('Asia/Ho_Chi_Minh')).strftime('%Y-%m-%d')
data.drop_duplicates(inplace=True)
data['Days_Aging'] = data['Aging'].dt.round("d").apply(lambda x: x.days)
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai,Days_Aging
0,9FIXAXS1K,S1805983O1345,2020-09-11 09:50:29+07:00,2020-09-11 09:50:29+07:00,2020-09-20 19:04:48+07:00,2020-09-23 07:42:31+07:00,NaT,2020-09-25 00:32:00+07:00,NaT,NaN,Quận Hai Bà Trưng,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Lưu kho,10,10x5x10,2516210,'23/09/2020 02:30:30 - GHN-DFC1A7|THAY ĐỔI THÔ...,'',2 Ngày,1,1,0,NaN,1714295,NaN,'Just Collection',1262603,20NDN45868,2329.0,2275.0,2020-09-22 20:45:08+07:00,Kết thúc,Kho giao,2020-09-22 20:45:08+07:00,2020-09-27 20:45:08+07:00,0 days 05:05:23.235187,Others,16 days 16:00:02.902446,2020-09-28,0
1,9FIFNX7NS4,5f5ee464ef9ddc00013f4d8a_476,2020-09-14 10:53:38+07:00,2020-09-14 10:53:38+07:00,2020-09-17 09:49:32+07:00,NaT,NaT,2020-09-21 23:00:00+07:00,NaT,NaN,Huyện Yên Mỹ,Hưng Yên,B,Quận 7,Hồ Chí Minh,E,Bưu Cục Yên Mỹ-Hưng Yên,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Yên Mỹ-Hưng Yên,Lưu kho,1000,10x80x10,2522198,NaN,'XEM HÀNG ĐÚNG K NHẬN TT 30K SHIP_riêng',3 Ngày,1,0,0,NaN,2045279,NaN,'Shop Táo Đỏ Hàn Quốc ',1274345,20CXV54912,2268.0,2275.0,2020-09-19 23:21:29+07:00,Kết thúc,Kho giao,2020-09-19 23:21:29+07:00,2020-09-24 23:21:29+07:00,3 days 02:29:02.235187,Others,13 days 14:56:53.902446,2020-09-28,3
2,9FIAN3YLY4,5f5f1b1a8ffa090001ec1676_102,2020-09-14 14:34:37+07:00,2020-09-14 14:34:02+07:00,2020-09-15 21:02:16+07:00,2020-09-19 07:48:26+07:00,NaT,2020-09-18 23:00:00+07:00,NaT,NaN,Quận Nam Từ Liêm,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Kho Khách hàng lớn Hà Nội,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hà Nội,Giao hàng không thành công,2000,20x30x10,1219299,'19/09/2020 04:16:05 - GHN-DFC1A4|KHÔNG LIÊN L...,'',2 Ngày,0,4,0,NaN,NaN,NaN,'Sách Online',1296420,20LRF06868,2329.0,2275.0,2020-09-18 14:19:07+07:00,Kết thúc,Kho giao,2020-09-18 14:19:07+07:00,2020-09-23 14:19:07+07:00,4 days 11:31:24.235187,Others,13 days 11:15:54.902446,2020-09-28,4
3,9FIHY13R57,SDT/2020/10559,2020-09-14 15:58:29+07:00,2020-09-14 15:58:29+07:00,2020-09-14 19:30:33+07:00,2020-09-16 09:05:36+07:00,NaT,2020-09-17 08:18:00+07:00,NaT,NaN,Quận 8,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 2999 Phạm Thế Hiển-Q.8-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 2999 Phạm Thế Hiển-Q.8-HCM,Lưu kho,500,10x10x10,1270353,'16/09/2020 12:21:12 - GHN-DFC1A0|THAY ĐỔI THÔ...,'2 TÚI MT01 SNM; CHO KHÁCH XEM HÀNG NẾU YÊU CẦ...,1 Ngày,1,2,0,NaN,2038040,NaN,'Shop Dạ Thảo',1256302,20EPQ98773,2329.0,2275.0,2020-09-15 01:10:34+07:00,Kết thúc,Kho giao,2020-09-15 01:10:34+07:00,2020-09-20 01:10:34+07:00,8 days 00:39:57.235187,Others,13 days 09:52:02.902446,2020-09-28,8
4,9FIHY1FA1U,5f5f19958ffa090001ec1675_50,2020-09-14 14:26:45+07:00,2020-09-14 14:26:45+07:00,2020-09-14 19:30:12+07:00,2020-09-16 08:03:21+07:00,2020-09-25 09:27:15+07:00,2020-09-21 06:21:00+07:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Huyện Phù Cừ,Hưng Yên,B,Quận 4,Hồ Chí Minh,E,Bưu Cục 11 Chu Mạnh Trinh-TP.Hưng Yên,Bưu Cục Q.4-HCM,Bưu Cục Q.4-HCM,Bưu Cục 11 Chu Mạnh Trinh-TP.Hưng Yên,Chờ giao lại,400,18x20x6,2521655,'16/09/2020 10:47:16 - GHN-DFC1A0|THAY ĐỔI THÔ...,"'anh/chị cho khách xem hàng, được thử nước như...",3 Ngày,1,4,0,NaN,1872071,NaN,'TOP STORE',1254778,20GYL77773,2275.0,2471.0,2020-09-22 14:26:03+07:00,Kết thúc,Kho giao,2020-09-22 14:26:03+07:00,2020-09-27 14:26:03+07:00,0 days 11:24:28.235187,Others,13 days 11:23:46.90244

In [15]:
# export data backlog to csv file
# data export
data_to_export = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'ECommerces',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Days_Aging',
    'Aging_ToanTrinh',
    'TrangThaiLuanChuyen',
    'MaKien',
]]

data_to_export.to_csv('backlog.csv', encoding='utf-8-sig', index=False)


In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g


# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_to_export.loc[data_to_export['Aging'] >= pd.Timedelta(hours=0)]
# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [ ]:
# read dataframe from google sheet
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)

# df_gspread

In [ ]:
# Inventory report
# from df2gspread import df2gspread as df2g
data_inventory = data.loc[(data['LoaiBacklog'] == 'Kho giao') & (data['TrangThai'] != 'Đang giao hàng')]
data_inventory['LoaiXuLy'] = 'Chưa giao lại'
chuagiao_filter = data_inventory['ThoiGianGiaoLanDau'].isnull()
giaoloi_filter = ~data_inventory['GhiChuGHN'].isnull() & data_inventory['GhiChuGHN'].str.contains(datetime.today().strftime('%d/%m/%Y'))
data_inventory.loc[chuagiao_filter, 'LoaiXuLy'] = 'Chưa giao lần nào'
data_inventory.loc[giaoloi_filter, 'LoaiXuLy'] = 'Giao lỗi'

def split_day(x):
    try:
        return x.strftime('%Y-%m-%d')
    except ValueError:
        return np.nan

def split_time(x):
    try:
        return x.strftime('%H:%M')
    except ValueError:
        return np.nan


data_inventory['N_ve_kho'] = data_inventory['N0'].apply(lambda x: split_day(x))
data_inventory['H_ve_kho'] = data_inventory['N0'].apply(lambda x: split_time(x))

del data_inventory['N0']
del data_inventory['N+']
del data_inventory['Aging']
del data_inventory['NgayHienTai']
del data_inventory['GhiChu']
# data_inventory
# to csv file
# data_inventory.to_csv('rp_giao.csv', index=False, encoding='utf-8-sig')
# data_inventory.to_excel('report_giao.xlsx')

# pivot data and push to google sheet
table1 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], aggfunc='count')
table2 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], columns=['N_ve_kho'], aggfunc='count', fill_value=0)

# table1
WKS_INVENTORY = 'RPGiao_bot'
# inventory_wks = gc.open_by_key(SPREADSHEET_ID).worksheet(WKS_INVENTORY)
# # inventory_wks.clear()  # clear old data
df2g.upload(
    table1,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    start_cell='A1',
)

# inventory_wks.add_row(table2.shape[0])

df2g.upload(
    table2,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    clean=False,
    start_cell='A8',
)
# then write detail inventory data
inventory_upload = data_inventory[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanGiao',
    'ECommerces',
    'LoaiXuLy',
    'N_ve_kho',
    'H_ve_kho',
    
]]
# export to csv
inventory_upload.to_csv('rp_giao.csv', encoding='utf-8-sig', index=False)

df2g.upload(
    inventory_upload,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=False,
    col_names=True,
    clean=False,
    start_cell='A15',
)

In [ ]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val